# DEPENDENCIAS

Puedes ejecutar una vez este codigo y luego borrarlo.

# Librerias

In [1]:
#import requests
import pandas as pd
import re
import json
import os
from bs4 import BeautifulSoup as bs

import random

import time

import pandas as pd
#import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

import undetected_chromedriver as uc

from dotenv import load_dotenv
from openai import OpenAI
from pydantic import BaseModel




# Entradas

In [2]:
# Ruta local al driver de Chrome
ruta_driver = r"C:\Users\diego\OneDrive\Escritorio\PyCharm\Chambas\chromedriver-win64/chromedriver.exe"

# Configuración de opciones del navegador
chrome_options = Options()
# chrome_options.add_argument('--headless')  # Ejecutar sin abrir ventana
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--no-sandbox')

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Obtención de Enlaces

In [3]:
# Abre Google
service = Service(executable_path=ruta_driver)
browser = webdriver.Chrome(service=service, options=chrome_options)

# Esta variable solo servira para asignarle nombre al Excel que se generara al final.
# Te recomiendo ponerle algo con lo que lo diferencies de los demás.
busqueda = 'Software Architect'

# AQUI poner el enlace de GLASSDOOR con el trabajo ya buscado:
# Abres ESTE mismo enlace
# Buscas el trabajo que necesitas "Scrappear"
# Copias el enlace principal y lo pegas acá.
url = 'https://www.glassdoor.com.mx/Empleo/estados-unidos-software-architect-empleos-SRCH_IL.0,14_IN1_KO15,33.htm'

print(url)

# Se ingresa el URL asignado
browser.get(url)

# Obtención de estructura HTML
html = browser.page_source
soup = bs(html, 'lxml')

# Filtrado de enlaces
articles = soup.find('ul', {'class': 'JobsList_jobsList__lqjTr'}).find_all('a', {'class': 'JobCard_jobTitle__GLyJ1'})
link_chambas = [article.get('href') for article in articles]

browser.quit()

# Selecciona 15 enlaces aleatorios de la lista
# Este lo puedes ajustar como gustes, yo puse 15 para tener de donde escoger.
link_chambas = random.sample(link_chambas, min(len(link_chambas), 10))
link_chambas



SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 137
Current browser version is 139.0.7258.66 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#sessionnotcreatedexception
Stacktrace:
	GetHandleVerifier [0x0x7ff63c5afe75+79173]
	GetHandleVerifier [0x0x7ff63c5afed0+79264]
	(No symbol) [0x0x7ff63c369e5a]
	(No symbol) [0x0x7ff63c3ae26f]
	(No symbol) [0x0x7ff63c3ad2bb]
	(No symbol) [0x0x7ff63c3a68ed]
	(No symbol) [0x0x7ff63c3a2b1d]
	(No symbol) [0x0x7ff63c3f67de]
	(No symbol) [0x0x7ff63c3f5f70]
	(No symbol) [0x0x7ff63c3e8743]
	(No symbol) [0x0x7ff63c3b14c1]
	(No symbol) [0x0x7ff63c3b2253]
	GetHandleVerifier [0x0x7ff63c87a2ad+3004797]
	GetHandleVerifier [0x0x7ff63c8746fd+2981325]
	GetHandleVerifier [0x0x7ff63c893350+3107360]
	GetHandleVerifier [0x0x7ff63c5ca9fe+188622]
	GetHandleVerifier

## Scrapping de Trabajos y Exportación EXCEL

In [149]:

data = []

for url in link_chambas:

    service = Service(executable_path=ruta_driver)
    browser = webdriver.Chrome(service=service, options=chrome_options)
    browser.get(url)

    wait_time = random.uniform(3, 6)
    print(f"Esperando {wait_time:.2f} segundos antes de continuar...")
    time.sleep(wait_time)

    soup = bs(browser.page_source, 'html.parser')

    vacante = {}

    try:
        info = soup.find('div',{'class':'JobDetails_jobDetailsContainer__y9P3L'})
        vacante["Descripción"] = info.get_text(separator='\n').strip()
    except AttributeError:
        print(f"No se encontró la info en: {url}")

    vacante["Enlace"] = url  # Esto no genera error, así que no necesitas try/except aquí

    data.append(vacante)
    print(vacante)

    print("")
    browser.quit()

# Eliminar fragmento de texto especifico que siempre sale.
for vacante in data:
    vacante["Descripción"] = vacante["Descripción"].replace("Postularse en el sitio del empleador\nPostúlate ahora", "")

# Guardar datos como JSON estructurado y legible
nombre_archivo = f"GlassDoor_{busqueda.replace(' ', '_')}.json"
with open(nombre_archivo, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print("Los datos se han exportado correctamente como JSON.")


Esperando 4.01 segundos antes de continuar...
{'Descripción': "GitLab\n4.1\nData System Architect, Data Engineering & Monetization New\nPostularse en el sitio del empleador\nPostúlate ahora\nRemote\nPostularse en el sitio del empleador\nPostúlate ahora\nGitLab is an open-core software company that develops the most comprehensive AI-powered DevSecOps Platform, used by more than 100,000 organizations. Our mission is to enable everyone to contribute to and co-create the software that powers our world. When everyone can contribute, consumers become contributors, significantly accelerating human progress. Our platform unites teams and organizations, breaking down barriers and redefining what's possible in software development. Thanks to products like Duo Enterprise and Duo Agent Platform, customers get AI benefits at every stage of the SDLC.\n\n\n\nThe same principles built into our products are reflected in how our team works: we embrace AI as a core productivity multiplier, with all team 

# Open AI USE

In [150]:
class structure(BaseModel):
    place: str
    company: str
    position: str
    academic_profile: str
    english_level: str
    technical_skills: str
    soft_skills: str
    salary: str
    source: str
    link: str

In [151]:

resultados = []
df = pd.DataFrame()
df_vacantes = pd.DataFrame(data)

columnas_estructuradas = structure.model_fields.keys()
for col in columnas_estructuradas:
    df_vacantes[col] = None

for idx, row in df_vacantes.iterrows():
    texto = row["Descripción"]
    enlace = row["Enlace"]
    print(f"Procesando vacante {idx+1} de {len(df_vacantes)}: {enlace}")


    try:
        response = client.responses.parse(
            model = "gpt-4.1-mini",
            input = [
                {
                    #Instrucciones para el asistente
                    "role": "system",
                    "content": "Eres un asistente experto en extraer información de ofertas laborales. Tu tarea es analizar el texto proporcionado y generar una estructura con los campos requeridos sin alterar el texto original (A menos que se te especifique, como es el caso de English_level). No debes incluir ningún comentario o explicación adicional, solo el contenido estructurado con la información extraída. En el campo de 'english_level', si no se menciona explicitamente pero se infiere que el puesto requiere un nivel de inglés avanzado (por ejemplo, que el pais de la solicitud sea angloparlante), debes asignar 'Nativo'. Si no se menciona el nivel de inglés, asigna 'No especificado'. El apartado de 'source' siempre sera 'Glassdoor'.",
                },
                {
                    "role": "user",
                    "content": f"Usa el siguiente texto para extraer la información requerida:\n\n{texto}, enlace: {enlace}"
                },
            ],
            temperature=0.2,
            text_format=structure,
        )
        
        resultados = pd.DataFrame(response.output_parsed)
        resultados = resultados.transpose() # Transponer el DataFrame
        resultados.columns = resultados.iloc[0] # Columna

        resultados = resultados[1:]  # Eliminar la primera fila que ahora es el encabezado
        df = pd.concat([df, resultados], ignore_index=True)
        print("Trabajos formateados exitosamente.")
        df
    


    except Exception as e:
        print(f"Error al procesar la vacante {enlace}")
        print(e)
    

Procesando vacante 1 de 10: https://www.glassdoor.com/job-listing/data-system-architect-data-engineering-monetization-new-gitlab-JV_KO0,55_KE56,62.htm?jl=1009834151298
Trabajos formateados exitosamente.
Procesando vacante 2 de 10: https://www.glassdoor.com/job-listing/backend-software-architect-python-echo-neurotechnologies-JV_IC1147401_KO0,33_KE34,56.htm?jl=1009816765091
Trabajos formateados exitosamente.
Procesando vacante 3 de 10: https://www.glassdoor.com/job-listing/software-architect-php-blockchain-blockskye-inc-JV_IC1154532_KO0,33_KE34,47.htm?jl=1009783516575
Trabajos formateados exitosamente.
Procesando vacante 4 de 10: https://www.glassdoor.com/job-listing/senior-software-architect-vettura-JV_IC1139761_KO0,25_KE26,33.htm?jl=1009788611942
Trabajos formateados exitosamente.
Procesando vacante 5 de 10: https://www.glassdoor.com/job-listing/data-architect-mapfre-puerto-rico-JV_IC1141599_KO0,14_KE15,33.htm?jl=1009827830017
Trabajos formateados exitosamente.
Procesando vacante 6 de 

---

In [155]:
df

,place,company,position,academic_profile,english_level,technical_skills,soft_skills,salary,source,link,"Atlanta, GA"
0,Remote,GitLab,"Data System Architect, Data Engineering & Mone...",No especificado,No especificado,Architecting large-scale distributed data syst...,"Leadership, team mentorship, collaborative val...","$157,900 - $236,900 USD (California/Colorado/H...",Glassdoor,https://www.glassdoor.com/job-listing/data-sys...,NaN
1,"San Francisco, CA",Echo Neurotechnologies,Backend Software Architect - Python,No especificado,No especificado,Extensive experience in building cloud-based s...,"Ability to take ownership of broad decisions, ...","Competitive compensation, including stock options",Glassdoor,https://www.glassdoor.com/job-listing/backend-...,NaN
2,"Boston, MA",BLOCKSKYE INC,Software Architect (PHP + Blockchain),"Bachelor’s degree in Computer Science, Softwar...",Avanzado,"Laravel, Node.js, AWS (Lambda, RDS, S3, ECS/Fa...","Curiosity, Discipline, Excellence, Thoughtfuln...",No especificado,Glassdoor,https://www.glassdoor.com/job-listing/software...,NaN
3,"Austin, TX",Vettura,Senior Software Architect,Master’s degree in Computer Science or related...,No especificado,"C#, Azure Cloud, DevOps, Azure-native services...",Excellent collaboration and communication skil...,No especificado,Glassdoor,https://www.glassdoor.com/job-listing/senior-s...,NaN
4,"San Juan, PR",MAPFRE PUERTO RICO,Data Architect,"Bachillerato en: Ciencias de Computadoras, Inf...",Bilingüe,"Experiencia avanzada en diseño, modelado y opt...",Capacidad para manejar múltiples proyectos sim...,"$5,000.00 - $6,000.00 per month, Bono Variable",Glassdoor,https://www.glassdoor.com/job-listing/data-arc...,NaN
5,"San Francisco, CA",UCSF Health,Data Warehouse Architect,Bachelor's degree in related area and / or equ...,No especificado,"Microsoft SQL Server, SSIS, Tableau BI or simi...",Interpersonal skills to work with technical an...,"$190,000 to $210,000 (Annual Rate)",Glassdoor,https://www.glassdoor.com/job-listing/data-war...,NaN
6,"Plano, TX",Innova Software Services,GenAI Architect,No especificado,No especificado,"Generative Models (LLMs, GANs, VAEs, transform...","Problem-solving skills, Collaboration with tea...","$120,000.00 - $140,000.00 per year",Glassdoor,https://www.glassdoor.com/job-listing/genai-ar...,NaN
7,"Arizona, AZ",Thermo Fisher Scientific,Bioinformatics Software Architect,PhD from an accredited higher learning institu...,No especificado,Experience developing analytical software tool...,"Pragmatic self-starter who is proactive, self-...",No especificado,Glassdoor,https://www.glassdoor.com/job-listing/bioinfor...,NaN
8,"Washington, DC",AARP,"Solution Architect, Generative Artificial Inte...",Bachelor’s or Master’s degree in Computer Scie...,No especificado,Cloud infrastructure management and architectu...,Effective verbal and written communication ski...,No especificado,Glassdoor,https://www.glassdoor.com/job-listing/solution...,NaN
9,"Atlanta, GA",eBay,Cloud Architect,A relevant bachelor’s degree and 8 years of re...,No especificado,"Expertise in systems architecture, including d...",Provide guidance and mentorship to engineering...,"$115,200 - $199,700",Glassdoor,https://www.glassdoor.com/job-listing/cloud-ar...,NaN


In [154]:
# Exportar datos a Excel
df.to_excel(f"sad{busqueda.replace(' ', '_')}.xlsx", index=False, engine="openpyxl")